In [1]:
import tensorflow as tf
from tensorflow.python.ops.rnn import _transpose_batch_time

from datetime import datetime
import numpy as np
import pandas as pd
import glob, os, sys, math, warnings
import matplotlib.pyplot as  plt
import copy, time, glob, os, sys
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# customized ftns 
from helpers import *
from utilities import *
from model import *
from train import train_all_single_policies
# ---------------------------------------------------------
%matplotlib inline
%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
# ---------------------------------------------------------
# directories
main_dir = '../'
game_dir = main_dir+'data/'
Data = LoadData(main_dir, game_dir)
models_path = './models/' 

C:\Users\sshi\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Load raw data

In [2]:
# %%time
game_id = '0021500196'
game_data = Data.load_game(game_id)
events_df = pd.DataFrame(game_data['events'])
print('raw events shape:', events_df.shape)
events_df.head(3)

raw events shape: (234, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,716.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...",[0],"Empty DataFrame Columns: [GAME_ID, EVENTNUM, E...",1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,701.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...","[1, 2]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,682.36,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224825036, 707.33, 11.37, None, [[-1,...","[3, 4]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,707.33,"{'abbreviation': 'TOR', 'players': [{'playerid..."


#### Get some suplementary data

In [3]:
# # play id to play roles/positions
# id_role = id_position(events_df)
# check_game_roles_duplicates(id_role)

# # its possible that F has similar role as G-f or F-G, we create empty slots to ensure meta order
# # ddentify defending and offending runs (this is included in process_moments)
# court_index = Data.load_csv('./meta_data/court_index.csv')
# court_index = dict(zip(court_index.game_id, court_index.court_position))

# # home and visitor ids
# homeid = events_df.loc[0].home['teamid']
# awayid = events_df.loc[0].visitor['teamid']

### FILTER EVENTS

In [4]:
# # filter out actions except 1: Make, 2: Miss, 4: Rebound, 6:Personal Foul, 7:Violation
# use_event = [1, 2]
# discard_event = [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 18]
# events = filter_event_type(events_df, discard_event)

In [5]:
events_df.shape

(234, 8)

In [6]:
events_df.head()

,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,716.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...",[0],"Empty DataFrame Columns: [GAME_ID, EVENTNUM, E...",1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,701.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...","[1, 2]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,682.36,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224825036, 707.33, 11.37, None, [[-1,...","[3, 4]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,707.33,"{'abbreviation': 'TOR', 'players': [{'playerid..."
3,670.36,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224844036, 688.32, 6.32, None, [[-1, ...","[5, 6]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,688.32,"{'abbreviation': 'TOR', 'players': [{'playerid..."
4,636.34,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224856036, 676.32, 13.32, None, [[-1,...",[7],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,676.32,"{'abbreviation': 'TOR', 'players': [{'playerid..."


In [26]:
events_df.loc[3].playbyplay.to_dict('list')

{'EVENTMSGACTIONTYPE': [1, 0, 1, 0],
 'EVENTMSGTYPE': [2, 4, 2, 4],
 'EVENTNUM': [4, 5, 6, 7],
 'GAME_ID': ['0021500196', '0021500196', '0021500196', '0021500196'],
 'HOMEDESCRIPTION': [None,
  'Jordan REBOUND (Off:0 Def:1)',
  "MISS Paul 26' 3PT Jump Shot",
  None],
 'NEUTRALDESCRIPTION': [None, None, None, None],
 'PCTIMESTRING': ['11:25', '11:24', '11:12', '11:11'],
 'PERIOD': [1, 1, 1, 1],
 'PERSON1TYPE': [5.0, 4.0, 4.0, 5.0],
 'PERSON2TYPE': [0, 0, 0, 0],
 'PERSON3TYPE': [0, 0, 0, 0],
 'PLAYER1_ID': [200768, 201599, 101108, 2449],
 'PLAYER1_NAME': ['Kyle Lowry', 'DeAndre Jordan', 'Chris Paul', 'Luis Scola'],
 'PLAYER1_TEAM_CITY': ['Toronto', 'LA', 'LA', 'Toronto'],
 'PLAYER1_TEAM_ID': [1610612761.0, 1610612746.0, 1610612746.0, 1610612761.0],
 'PLAYER1_TEAM_NICKNAME': ['Raptors', 'Clippers', 'Clippers', 'Raptors'],
 'PLAYER2_ID': [0, 0, 0, 0],
 'PLAYER2_NAME': [None, None, None, None],
 'PLAYER2_TEAM_ABBREVIATION': [None, None, None, None],
 'PLAYER2_TEAM_CITY': [None, None, None, 

In [23]:
events_df.moments[3]

[[1,
  1448224844036,
  688.32,
  6.32,
  None,
  [[-1, -1, 79.7451, 22.04133, 15.88121],
   [1610612746, 1718, 84.66993, 30.72191, 0.0],
   [1610612746, 200755, 76.44073, 30.87877, 0.0],
   [1610612746, 101108, 85.26235, 11.90967, 0.0],
   [1610612746, 201599, 89.71153, 32.2652, 0.0],
   [1610612746, 201933, 71.4998, 19.57765, 0.0],
   [1610612761, 2449, 84.0546, 7.85235, 0.0],
   [1610612761, 201960, 82.85868, 46.14123, 0.0],
   [1610612761, 200768, 67.44618, 18.94625, 0.0],
   [1610612761, 201942, 64.41625, 41.52674, 0.0],
   [1610612761, 202687, 88.05373, 37.41423, 0.0]]],
 [1,
  1448224844076,
  688.28,
  6.28,
  None,
  [[-1, -1, 80.33176, 22.3495, 15.93851],
   [1610612746, 1718, 84.60024, 30.50806, 0.0],
   [1610612746, 200755, 76.50204, 30.7477, 0.0],
   [1610612746, 101108, 85.08484, 12.01026, 0.0],
   [1610612746, 201599, 89.6397, 32.20678, 0.0],
   [1610612746, 201933, 71.30691, 19.6634, 0.0],
   [1610612761, 2449, 83.80019, 8.20592, 0.0],
   [1610612761, 201960, 82.70972, 

In [17]:
events_df.dropna(subset=['playbyplay'], inplace=True)
events_df.reset_index(drop=True, inplace=True)

use_events = [1,2]
def filter_events(row, use_events):
    pbp = row['playbyplay'].copy()
    # when theres no playbyplay data
    if len(pbp) == 0:
        return []
    time_lefts = pbp['time_left'].values
    n_events = len(pbp)
    result = []
    event_i = []
    init = 0
    start_time = time_lefts[init]
    for i in row['moments']:
        if i[2] > start_time:
            event_i.append(i)
        else:
            result.append(event_i)
            if init < n_events - 1:
                init += 1
                start_time = time_lefts[init]
                event_i = []
    select_events = [result[i] for i in range(n_events) if pbp['EVENTMSGTYPE'].values[i] in use_events]
    return select_events

In [18]:
events_df['n'] = events_df.apply(lambda row: filter_events(row, use_events), axis=1)

In [19]:
events_df.n[2]

[[[1,
   1448224825036,
   707.33,
   11.37,
   None,
   [[-1, -1, 8.0386, 25.71911, 12.20696],
    [1610612746, 1718, 31.26335, 9.90733, 0.0],
    [1610612746, 200755, 28.84064, 27.58334, 0.0],
    [1610612746, 101108, 17.1999, 5.2689, 0.0],
    [1610612746, 201599, 31.20443, 20.28944, 0.0],
    [1610612746, 201933, 21.19991, 41.58761, 0.0],
    [1610612761, 2449, 11.80706, 30.66015, 0.0],
    [1610612761, 201960, 23.26725, 25.25622, 0.0],
    [1610612761, 200768, 8.49091, 18.21114, 0.0],
    [1610612761, 201942, 19.46496, 23.96676, 0.0],
    [1610612761, 202687, 12.82137, 23.6505, 0.0]]],
  [1,
   1448224825077,
   707.29,
   11.33,
   None,
   [[-1, -1, 7.35226, 25.51223, 11.72727],
    [1610612746, 1718, 31.45588, 9.91133, 0.0],
    [1610612746, 200755, 29.06116, 27.46613, 0.0],
    [1610612746, 101108, 17.3095, 5.45749, 0.0],
    [1610612746, 201599, 31.55956, 20.27081, 0.0],
    [1610612746, 201933, 21.45894, 41.52586, 0.0],
    [1610612761, 2449, 11.69035, 30.50215, 0.0],
    [1

In [ ]:
events_df.head()

In [ ]:
events_df.playbyplay[2]

In [15]:
len(events_df.moments[3])

450

In [16]:
P = PlotGame('0021500196', main_dir, game_dir)
for i in range(450):
    P.load_moment2img(game_data, 3, i)

In [12]:
images = glob.glob('../data/game0021500196/event2/' + "*.png")
images = sorted(images, key=lambda x: int(x.split('.')[2].split('/')[-1]))
make_video(images, '../data/game0021500196/event2/2.mp4', 15)

ValueError: invalid literal for int() with base 10: 'event2\\0'